## Random cutout

In [2]:
import os
import cv2
import numpy as np
import random

# Define the directory where your dataset images and annotations are stored
dataset_dir = 'F:/Minor 5/arvalis_1/image'
output_dir = 'F:/Minor 5/arvalis_1/masked'
annotations_dir = 'F:/Minor 5/arvalis_1/label'  # Directory containing YOLO format annotation files

# Define the size of the mask and the number of random images you want to process
mask_size = (190,40)
num_images_to_process = 505
#num_masks_per_image = 7
max_rotation_angle = 90
mask_color = (0, 255, 0)

# Function to extract bounding box centers from the YOLO format annotation file
def extract_bounding_box_centers(annotation_file_path):
    centers = []
    with open(annotation_file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split(' ')
            label = int(parts[0])  # assuming label is the first element
            x_center, y_center, width, height = map(float, parts[1:5])

            # Convert YOLO format to bounding box coordinates (xmin, ymin, xmax, ymax)
            xmin = int((x_center - width / 2) * image_width)
            ymin = int((y_center - height / 2) * image_height)
            xmax = int((x_center + width / 2) * image_width)
            ymax = int((y_center + height / 2) * image_height)

            center_x = (xmin + xmax) // 2
            center_y = (ymin + ymax) // 2

            centers.append((center_x, center_y))

    return centers

# Randomly select and process a subset of images
selected_images = random.sample(os.listdir(dataset_dir), num_images_to_process)

for image_file in selected_images:
    image_path = os.path.join(dataset_dir, image_file)
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    annotation_file_path = os.path.join(annotations_dir, image_file.replace('.JPG', '.txt'))

    if os.path.exists(annotation_file_path):
        bounding_box_centers = extract_bounding_box_centers(annotation_file_path)
        num_masks_per_image = int(len(bounding_box_centers) * 0.6)
        #for _ in range(num_masks_per_image):
        if bounding_box_centers:
                selected_centers = random.sample(bounding_box_centers, min(num_masks_per_image, len(bounding_box_centers)))

                modified_image = image.copy()

                for center_x, center_y in selected_centers:
                    mask = np.zeros_like(image)
                    mask_center_x, mask_center_y = mask_size[0] // 2, mask_size[1] // 2
                    mask_start_x, mask_start_y = max(center_x - mask_center_x, 0), max(center_y - mask_center_y, 0)
                    mask_end_x, mask_end_y = min(center_x + mask_center_x, image_width), min(center_y + mask_center_y, image_height)

                    mask[mask_start_y:mask_end_y, mask_start_x:mask_end_x] = mask_color
                    rotation_angle = random.uniform(-max_rotation_angle, max_rotation_angle)
                    rotation_matrix = cv2.getRotationMatrix2D((center_x, center_y), rotation_angle, 1)
                    rotated_mask = cv2.warpAffine(mask, rotation_matrix, (image_width, image_height))

            # Apply the rotated mask to the modified image

                # Apply the mask to the modified image
                    modified_image = cv2.addWeighted(modified_image, 1, rotated_mask, 1, 0)

                # Save the modified image
                output_file = os.path.join(output_dir, image_file)
                cv2.imwrite(output_file, modified_image)
        os.remove(image_path)

## SPLITTING


In [12]:
import os
from random import choice
import shutil

#arrays to store file names
imgs =[]
xmls =[]

#setup dir names
trainPath = 'D:/Minor 5/YOLOV8/Dataset/images/train'
valPath = 'D:/Minor 5/YOLOV8/Dataset/images//val'
crsPath = 'F:/Minor 5/global/combgreen' #dir where images and annotations stored

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)


#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(imgs)*val_ratio)
print("training images are : ",countForTrain)
print("Validation images are : ",countForVal)

training images are :  1008
Validation images are :  252


In [13]:
trainimagePath = 'F:/Minor 5/YOLOV8/Dataset/train/images'
trainlabelPath = 'F:/Minor 5/YOLOV8/Dataset/train/labels'
valimagePath = 'F:/Minor 5/YOLOV8/Dataset/val/images'
vallabelPath = 'F:/Minor 5/YOLOV8/Dataset/val/labels'
#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))


    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)



#cycle for test dir   
for x in range(countForVal):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    
    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)
shutil.move(crsPath, valPath) 

'D:/Minor 5/YOLOV8/Dataset/images//val\\combgreen'

## training

In [ ]:
from ultralytics import YOLO

In [ ]:
model2 = YOLO('yolov8x.pt')
model2.train(data="dataset5.yaml", epochs = 80, patience=60,cos_lr= True, dropout= 0.2, plots= True)